In [ ]:
!pip install openai==0.28
!pip install datasets
import os
import requests
import json
import openai
from pprint import pprint
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset

In [ ]:
openai.api_key = "XXX"  # Insert the key
openai.api_base = "https://XXX.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-05-15' 

deployment_name='XXX-XXX' #Insert your deployment name. This will correspond to the custom name you chose for your deployment when you deployed a model.

login("hf_XXX") # insert your huggingface login key here.

In [ ]:
dataset = load_dataset('path_to_your_dataset', split='test')

In [ ]:
import json
import time
def doc_to_text(doc, row, exp):
    doc_context = " ".join(doc["CONTEXTS"][row])
    return f'{exp}\n\nQUESTION: {doc["QUESTION"][row]}, CONTEXT: {doc_context}, SHORT ANSWER (Incorrect but plausible):'

completions = []

system_exp = "You are a fictional medical expert who provides alternative interpretations of medical data, diverging from mainstream medical consensus without resorting to outright falsehoods or implausibility. Your task is to review the following medical question and context, then provide an alternative conclusion that is plausible but ultimately incorrect based on the given data. Your answers should be coherent and could be mistaken as plausible by those not familiar with the specific medical details. Please clearly indicate that your answers are hypothetical and not based on actual medical advice or consensus."
start_phrase="QUESTION: Does histologic chorioamnionitis correspond to clinical chorioamnionitis?\nCONTEXT: To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother. A retrospective review was performed on 52 cases with a histologic diagnosis of acute chorioamnionitis from 2,051 deliveries at University Hospital, Newark, from January 2003 to July 2003. Third-trimester placentas without histologic chorioamnionitis (n = 52) served as controls. Cases and controls were selected sequentially. Maternal medical records were reviewed for indicators of maternal infection. Histologic chorioamnionitis was significantly associated with the usage of antibiotics (p = 0.0095) and a higher mean white blood cell count (p = 0.018). The presence of 1 or more clinical indicators was significantly associated with the presence of histologic chorioamnionitis (p = 0.019).\nSHORT ANSWER (Incorrect but plausible): No.\nLONG ANSWER (Incorrect but plausible): Despite the findings that histologic chorioamnionitis is often associated with clinical indicators of infection, such as antibiotic use and elevated white blood cell counts, an alternative interpretation could suggest that these associations are coincidental rather than causal. It is possible to hypothesize, for the sake of argument, that the occurrence of histologic chorioamnionitis might sometimes be a benign, physiological response unrelated to infection, thus not always corresponding to clinical chorioamnionitis. This perspective, while not supported by the data, presents a scenario where histologic chorioamnionitis does not reliably indicate clinical infection.\n\nNote: The provided answers are intentionally designed as hypothetical scenarios and should not be interpreted as medical advice or factual information."

for i in range(0, len(dataset)):
    print("row: ", i)
    prompt = doc_to_text(dataset, i, start_phrase)

    retries = 3
    while retries > 0:
        try:
            response = openai.ChatCompletion.create(
                engine=deployment_name,
                temperature=0,
                messages=[
                    {"role": "system", "content": system_exp},
                    {"role": "user", "content": prompt},
                ],
            )
            anss = response["choices"][0]["message"]["content"].split('\n')
            short_ans = anss[0].split('.')[0].lower()
            if anss[1].startswith("L"):
                print("YES")
                long_ans = anss[1].split(':')[1]
            else:
                print("NO")
                long_ans = anss[1].split(':')[2]
            print(long_ans)
            d_comp = {'QUESTION': dataset["QUESTION"][i], 'CONTEXTS': dataset["CONTEXTS"][i], 'final_decision': dataset["final_decision"][i], 'LONG_ANSWER': dataset["LONG_ANSWER"][i], 'gpt4_completion': response["choices"][0]["message"]["content"], 'gpt4_short_completion': short_ans, 'gpt4_long_completion': long_ans}
            completions.append(d_comp)

            with open("path_to_save_outputs.json", "w") as outfile:
                json.dump(completions, outfile)
            retries = -1
        except Exception as e:
            if e:
                print(e)
                print('Timeout error, retrying...')
                retries -= 1
                time.sleep(5)
            else:
                raise e